In [1]:
# libraries needed 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import re
%matplotlib inline

# libraries we might not need
import csv

### Using the following code
to use the whole document you only need one file specified by filepath for the time being

In [16]:
%%time
# imports a random sample of size s from csv-file as a pandas dataframe
# pandas using python 3.X uses utf-8 encoding

# usage: specify file location, sample size and seed(used by random)
filepath = '../Data_git_ignore/clean_corpus_token.csv'
#filepath = 'news_sample.csv' # <- overwrite for setup
s = 12000                    # desired sample size(seems to have slack ie. not exact)
seed = 1                     # seed used by Pseudorandom number generator

# init dataframe with specified values
df = pd.read_csv(filepath, index_col = [0]).sample(n=s, random_state=seed)

# visual output
#print(df.shape, '<- size of dataframe \n')
df.head()

Wall time: 828 ms


,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source,type_id,content_tokenize
3510,928838.0,christianpost.com,reliable,https://www.christianpost.com/news/saving-demo...,expand collapse photo reutersmohamed abd el g...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Saving Democracy From Itself,NaN,NaN,NaN,Every democracy must defend itself against tho...,NaN,NaN,NaN,7,"['expand', 'collapse', 'photo', 'reutersmohame..."
4316,929807.0,christianpost.com,reliable,https://www.christianpost.com/news/iran-backed...,photo reuterskhalid al mousily residents inspe...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Iran-Backed Militias Lead Attacks on Christian...,NaN,NaN,NaN,Radical militias with Iranian support are repo...,NaN,NaN,NaN,7,"['photo', 'reuterskhalid', 'al', 'mousily', 'r..."
4844,930439.0,christianpost.com,reliable,https://www.christianpost.com/news/dont-tear-d...,expand collapse photo reutersjason miczek the...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Don't Tear Down the Confederate Battle Flag,NaN,NaN,NaN,"Like many Southern boys, I grew up with two fl...",NaN,NaN,NaN,7,"['expand', 'collapse', 'photo', 'reutersjason'..."
2580,612104.0,beforeitsnews.com,fake,http://beforeitsnews.com/opinion-conservative/...,donald trump is in NUM gop nominee for presi...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump Is In 2016 GOP Nominee for President,American Freedom Barabara,NaN,NaN,NaN,NaN,NaN,NaN,3,"['donald', 'trump', 'is', 'in', 'NUM', 'gop', ..."
3432,469781.0,nutritionfacts.org,reliable,https://nutritionfacts.org/es/topics/salud-rep...,por qu las mujeres veganas tienen 5 veces meno...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,salud reproductiva,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,"['por', 'qu', 'las', 'mujeres', 'veganas', 'ti..."


In [17]:
df['id'] = pd.to_numeric(df['id'], errors = 'coerce', downcast = 'integer')
df.drop_duplicates(subset = 'id', inplace = True)

In [18]:
df = df.dropna(subset=['id']).set_index('id')
df.shape

(12000, 17)

### data-tables: [name]-uniq / relational-tables: [name]_in
creating csv-files for database

In [19]:
# specify where to save all csv-files
path = '../Data_git_ignore/clean_csv/'

# create temporary dataframe and use article id as index 
out_df = pd.DataFrame({'id':df.index})
out_df.set_index('id', inplace=True)

In [20]:
def generateUniqe (category):
    type_array = df[category].unique() # get array of unique types
    type_df = pd.DataFrame({'id': np.arange(type_array.size), 'name':type_array})

    # write file and free memory
    type_df.to_csv(path + category + '_clean.csv', index=False, header=True)
    #del type_array
    #del type_df # tmp delete later

    # create dict with type_name as key - [swap type with type_id]
    type_name_as_key_df = type_df.set_index('name')
    type_dict = type_name_as_key_df['id'].to_dict()

    # replace type with tag id and create new column
    type_id = np.array([type_dict[key] for key in df[category].to_numpy()])
    #df['type_id'] =type_id
    
    return type_id

def generateUniqeArray (category):
    # creates list of list but formaly it is a pd.series of lists
    tags_series_of_lists = df[category].dropna().str.split(', ') # -> ', ' not ','

    if not category in out_df: ### tmp need another method ###
        out_df.insert(0,column = category, value = tags_series_of_lists)

    # flattern tags_series_of_lists to a set(ie. unique values only)
    tags_list = list(set([item for sublist in tags_series_of_lists for item in sublist]))

    # create dataframe
    tags_df = pd.DataFrame({'id': np.arange(len(tags_list)), 'name':tags_list})

    # write file and free memory
    tags_df.to_csv(path + category + '_clean.csv', index=False, header=True)
    del tags_series_of_lists
    del tags_list
    
    # generate the relation
    # get all pairs of article_id and tags in a article (for all articles)
    articles_id_tags_name_pairs_df = out_df[category].dropna().explode().drop_duplicates(keep = 'first')

    # split tags_name and articles_id
    articles_id_array = articles_id_tags_name_pairs_df.index.to_numpy()
    tags_name_array = articles_id_tags_name_pairs_df.to_numpy()

    # create dict with tag_name as key - [swap tags with tags_id]
    tags_name_as_key_df = tags_df.set_index('name')
    tags_dict = tags_name_as_key_df['id'].to_dict()

    # replace tags with tag id
    tags_id = np.array([tags_dict[key] for key in tags_name_array])

    # create dataframe
    tags_in_df = pd.DataFrame(data=articles_id_array, index=tags_id, columns=['article_id'])
    tags_in_df.index.name=category + '_id'

    # write file and free memory
    tags_in_df.to_csv(path + category + '_in.csv', index=True, header=True)

In [21]:
### type_clean - data-table ###
df["type"] = generateUniqe("type")

In [22]:
### tags_clean & tags_in - data-table ###
generateUniqeArray("tags")

In [23]:
### authors_clean & authors_in - data-table ###
generateUniqeArray("authors")

In [24]:
### domain_clean - data-table ###
df["domain"] = generateUniqe("domain")

In [25]:
### meta_keywords_clean & meta_keywords_in - data-table ###
generateUniqeArray("meta_keywords")

In [30]:
# Clean meta keywords representation
regexStrList = r"\["
df['meta_keywords'] = df['meta_keywords'].replace(to_replace=regexStrList, value='{', regex=True)
regexStrList = r"\]"
df['meta_keywords'] = df['meta_keywords'].replace(to_replace=regexStrList, value='}', regex=True)

regexStrList = r"\["
df['content_tokenize'] = df['content_tokenize'].replace(to_replace=regexStrList, value='{', regex=True)
regexStrList = r"\]"
df['content_tokenize'] = df['content_tokenize'].replace(to_replace=regexStrList, value='}', regex=True)

df['content_tokenize']

id
928838    {'expand', 'collapse', 'photo', 'reutersmohame...
929807    {'photo', 'reuterskhalid', 'al', 'mousily', 'r...
930439    {'expand', 'collapse', 'photo', 'reutersjason'...
612104    {'donald', 'trump', 'is', 'in', 'NUM', 'gop', ...
469781    {'por', 'qu', 'las', 'mujeres', 'veganas', 'ti...
                                ...                        
157090    {'photo', 'youtubejohnson', 'johnsonscreengrab...
79885     {'pope', 'recalls', 'the', 'inspiring', 'witne...
931563    {'i', 'am', 'praying', 'not', 'only', 'for', '...
309432    {'bone', 'therapeutic', 'market', 'research', ...
333786    {'the', 'page', 'has', 'turned', 'to', 'a', 'n...
Name: content_tokenize, Length: 12000, dtype: object

In [31]:
df.head()

,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source,type_id,content_tokenize
id,,,,,,,,,,,,,,,,,
928838,0,0,https://www.christianpost.com/news/saving-demo...,expand collapse photo reutersmohamed abd el g...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Saving Democracy From Itself,NaN,NaN,NaN,Every democracy must defend itself against tho...,NaN,NaN,NaN,7,"{'expand', 'collapse', 'photo', 'reutersmohame..."
929807,0,0,https://www.christianpost.com/news/iran-backed...,photo reuterskhalid al mousily residents inspe...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Iran-Backed Militias Lead Attacks on Christian...,NaN,NaN,NaN,Radical militias with Iranian support are repo...,NaN,NaN,NaN,7,"{'photo', 'reuterskhalid', 'al', 'mousily', 'r..."
930439,0,0,https://www.christianpost.com/news/dont-tear-d...,expand collapse photo reutersjason miczek the...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Don't Tear Down the Confederate Battle Flag,NaN,NaN,NaN,"Like many Southern boys, I grew up with two fl...",NaN,NaN,NaN,7,"{'expand', 'collapse', 'photo', 'reutersjason'..."
612104,1,1,http://beforeitsnews.com/opinion-conservative/...,donald trump is in NUM gop nominee for presi...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump Is In 2016 GOP Nominee for President,American Freedom Barabara,NaN,NaN,NaN,NaN,NaN,NaN,3,"{'donald', 'trump', 'is', 'in', 'NUM', 'gop', ..."
469781,2,0,https://nutritionfacts.org/es/topics/salud-rep...,por qu las mujeres veganas tienen 5 veces meno...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,salud reproductiva,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,"{'por', 'qu', 'las', 'mujeres', 'veganas', 'ti..."


In [32]:
### article clean ###

df[['domain', 'type', "url", "content_tokenize", "title", "meta_keywords", "scraped_at",  "updated_at", "inserted_at"]].to_csv(path + 'article_clean.csv', header=True)

In [14]:
### Setting up wikinews

In [15]:
# imports a random sample of size s from csv-file as a pandas dataframe
# pandas using python 3.X uses utf-8 encoding

# usage: specify file location, sample size and seed(used by random)
filepath = '../Data_git_ignore/wiki_news_raw_backup_clean.csv'
#filepath = 'news_sample.csv' # <- overwrite for setup
s = 3679                    # desired sample size(seems to have slack ie. not exact)
seed = 1                     # seed used by Pseudorandom number generator

# init dataframe with specified values
df = pd.read_csv(filepath, index_col = [0]).sample(n=s, random_state=seed)

# visual output
#print(df.shape, '<- size of dataframe \n')
df.head()

,categories,content,publish_date,scraped_at,source_urls,sources_domain_urls,title
article_url,,,,,,,
https://en.wikinews.org/wiki/Royal_Canadian_Legion_upset_over_white_poppies,"November 9, 2006,Published,Archived,North Amer...","['\r', '', 'the', '', 'royal', 'canadian', 'le...","November 9, 2006",2020-04-22,http://www.cbc.ca/canada/calgary/story/2006/11...,"/wiki/CBC,https://en.wikipedia.org/wiki/Globe_...",Royal Canadian Legion upset over white poppies
https://en.wikinews.org/wiki/North_Korea_denies_cooperation_with_Iran_on_nuclear_programs,"January 27, 2007,Published,Archived,Korean Pen...","['\r', '', 'the', '', 'north', 'korean', '', '...","January 27, 2007",2020-04-22,http://www.cnn.com/2007/WORLD/asiapcf/01/27/ko...,"/wiki/CNN,https://en.wikipedia.org/wiki/Jerusa...",North Korea denies cooperation with Iran on nu...
https://en.wikinews.org/wiki/Rwandan_Senate_approves_constitutional_change_allowing_third_term_for_Kagame,"November 17, 2015,Published,Archived,Africa,Rw...","['\r', '', 'the', '', '', 'of', '', 'rwanda', ...","November 17, 2015",2020-04-22,http://www.news24.com/Africa/News/kagame-may-r...,"https://en.wikipedia.org/wiki/News24,/wiki/Reu...",Rwandan Senate approves constitutional change ...
https://en.wikinews.org/wiki/Traders%27_one_day_strike_in_Indian_capital,"November 7, 2006,Published,Archived,Politics a...","['\r', '', 'angry', 'traders', 'continued', 't...","November 7, 2006",2020-04-22,http://news.bbc.co.uk/2/hi/south_asia/6123252....,"/wiki/BBC_News_Online,/wiki/CNN-IBN,https://en...",Traders' one day strike in Indian capital
https://en.wikinews.org/wiki/Toilet_paper_running_thin_in_New_Jersey_capital,"New Jersey,March 13, 2012,Published,Archived,W...","['\r', '', 'toilet', 'paper', 'supplies', 'in'...","March 13, 2012",2020-04-22,http://www.bbc.co.uk/news/world-us-canada-1734...,"/wiki/BBC_News,https://en.wikipedia.org/wiki/T...",Toilet paper running thin in New Jersey capital


In [16]:
df['id'] = pd.to_numeric(range(10000000, 10000000+df.shape[0]), errors = 'coerce', downcast = 'integer')
df.drop_duplicates(subset = 'id', inplace = True)

df["category"] = 1

df = df.dropna(subset=['id']).set_index('id')
df.head()

,categories,content,publish_date,scraped_at,source_urls,sources_domain_urls,title,category
id,,,,,,,,
10000000,"November 9, 2006,Published,Archived,North Amer...","['\r', '', 'the', '', 'royal', 'canadian', 'le...","November 9, 2006",2020-04-22,http://www.cbc.ca/canada/calgary/story/2006/11...,"/wiki/CBC,https://en.wikipedia.org/wiki/Globe_...",Royal Canadian Legion upset over white poppies,1
10000001,"January 27, 2007,Published,Archived,Korean Pen...","['\r', '', 'the', '', 'north', 'korean', '', '...","January 27, 2007",2020-04-22,http://www.cnn.com/2007/WORLD/asiapcf/01/27/ko...,"/wiki/CNN,https://en.wikipedia.org/wiki/Jerusa...",North Korea denies cooperation with Iran on nu...,1
10000002,"November 17, 2015,Published,Archived,Africa,Rw...","['\r', '', 'the', '', '', 'of', '', 'rwanda', ...","November 17, 2015",2020-04-22,http://www.news24.com/Africa/News/kagame-may-r...,"https://en.wikipedia.org/wiki/News24,/wiki/Reu...",Rwandan Senate approves constitutional change ...,1
10000003,"November 7, 2006,Published,Archived,Politics a...","['\r', '', 'angry', 'traders', 'continued', 't...","November 7, 2006",2020-04-22,http://news.bbc.co.uk/2/hi/south_asia/6123252....,"/wiki/BBC_News_Online,/wiki/CNN-IBN,https://en...",Traders' one day strike in Indian capital,1
10000004,"New Jersey,March 13, 2012,Published,Archived,W...","['\r', '', 'toilet', 'paper', 'supplies', 'in'...","March 13, 2012",2020-04-22,http://www.bbc.co.uk/news/world-us-canada-1734...,"/wiki/BBC_News,https://en.wikipedia.org/wiki/T...",Toilet paper running thin in New Jersey capital,1


In [17]:
df.index

Int64Index([10000000, 10000001, 10000002, 10000003, 10000004, 10000005,
            10000006, 10000007, 10000008, 10000009,
            ...
            10003669, 10003670, 10003671, 10003672, 10003673, 10003674,
            10003675, 10003676, 10003677, 10003678],
           dtype='int64', name='id', length=3679)

In [18]:
path = '../Data_git_ignore/clean_csv/'

# create temporary dataframe and use article id as index 
out_df = pd.DataFrame({'id':df.index})
out_df.set_index('id', inplace=True)

In [19]:
def generateUniqe (category):
    type_array = df[category].unique() # get array of unique types
    type_df = pd.DataFrame({'id': np.arange(type_array.size), 'name':type_array})

    # write file and free memory
    type_df.to_csv(path + category + '_clean.csv', index=False, header=True)
    #del type_array
    #del type_df # tmp delete later

    # create dict with type_name as key - [swap type with type_id]
    type_name_as_key_df = type_df.set_index('name')
    type_dict = type_name_as_key_df['id'].to_dict()

    # replace type with tag id and create new column
    type_id = np.array([type_dict[key] for key in df[category].to_numpy()])
    #df['type_id'] =type_id
    
    return type_id

def generateUniqeArray (category):
    # creates list of list but formaly it is a pd.series of lists
    tags_series_of_lists = df[category].dropna().str.split(', ') # -> ', ' not ','

    if not category in out_df: ### tmp need another method ###
        out_df.insert(0,column = category, value = tags_series_of_lists)

    # flattern tags_series_of_lists to a set(ie. unique values only)
    tags_list = list(set([item for sublist in tags_series_of_lists for item in sublist]))

    # create dataframe
    tags_df = pd.DataFrame({'id': np.arange(len(tags_list)), 'name':tags_list})

    # write file and free memory
    tags_df.to_csv(path + category + '_clean.csv', index=False, header=True)
    del tags_series_of_lists
    del tags_list
    
    # generate the relation
    # get all pairs of article_id and tags in a article (for all articles)
    articles_id_tags_name_pairs_df = out_df[category].dropna().explode().drop_duplicates(keep = 'first')

    # split tags_name and articles_id
    articles_id_array = articles_id_tags_name_pairs_df.index.to_numpy()
    tags_name_array = articles_id_tags_name_pairs_df.to_numpy()
    
    # create dict with tag_name as key - [swap tags with tags_id]
    tags_name_as_key_df = tags_df.set_index('name')
    tags_dict = tags_name_as_key_df['id'].to_dict()

    # replace tags with tag id
    tags_id = np.array([tags_dict[key] for key in tags_name_array])
    
    # create dataframe
    tags_in_df = pd.DataFrame(data=articles_id_array, index=tags_id, columns=['id'])
    tags_in_df.index.name=category + '_id'

    # write file and free memory
    tags_in_df.to_csv(path + category + '_in.csv', index=True, header=True)

In [20]:
### categoires_clean & categories_in ###
generateUniqeArray("categories")

In [21]:
### sources_domain_urls_clean & sources_domain_urls_in
generateUniqeArray("sources_domain_urls")

In [22]:
# Clean meta keywords representation
regexStrList = r"\["
df['content'] = df['content'].replace(to_replace=regexStrList, value='{', regex=True)
regexStrList = r"\]"
df['content'] = df['content'].replace(to_replace=regexStrList, value='}', regex=True)

In [23]:
### article clean ###
df[['content', 'category', "title", "scraped_at", "publish_date"]].to_csv(path + 'wiki_article_clean.csv', header=True)

In [24]:
df[['content', 'category', "title", "scraped_at", "publish_date"]]

,content,category,title,scraped_at,publish_date
id,,,,,
10000000,"{'\r', '', 'the', '', 'royal', 'canadian', 'le...",1,Royal Canadian Legion upset over white poppies,2020-04-22,"November 9, 2006"
10000001,"{'\r', '', 'the', '', 'north', 'korean', '', '...",1,North Korea denies cooperation with Iran on nu...,2020-04-22,"January 27, 2007"
10000002,"{'\r', '', 'the', '', '', 'of', '', 'rwanda', ...",1,Rwandan Senate approves constitutional change ...,2020-04-22,"November 17, 2015"
10000003,"{'\r', '', 'angry', 'traders', 'continued', 't...",1,Traders' one day strike in Indian capital,2020-04-22,"November 7, 2006"
10000004,"{'\r', '', 'toilet', 'paper', 'supplies', 'in'...",1,Toilet paper running thin in New Jersey capital,2020-04-22,"March 13, 2012"
...,...,...,...,...,...
10003674,"{'\r', '', 'representatives', 'of', 'the', '',...",1,UN Security Council heads to Haiti amid politi...,2020-04-22,"January 24, 2015"
10003675,"{'\r', '', 'ismail', 'haniyeh', '', 'the', '',...",1,Palestinian PM: Israel aims to topple government,2020-04-22,"June 30, 2006"
10003676,"{'\r', '', 'iraq', 'we', 'have', 'declared', '...",1,Militant group declares war on Iraq poll,2020-04-22,"January 24, 2005"
